In [1]:
# total : 58 problem and solutions

In [2]:
# env : pixlake
# we focuing on pyspark dataframe processing
# documentation https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame
%load_ext autoreload
%autoreload 2

In [3]:
# make you auto compeletion faster
# https://stackoverflow.com/questions/40536560/ipython-and-jupyter-autocomplete-not-working
%config Completer.use_jedi = False

In [4]:
import os
import sys

def get_workstation_spark_path(where_are_you : str) -> str:
    if where_are_you == 'titan':
        return '/home/data/ryanchao2012/lib'
    elif where_are_you == 'thor':
        return '/opt/spark/versions'
    else:
        raise ValueError("wrong work station name")

spark_path = get_workstation_spark_path('thor')

print('You have pyspark version : ', os.listdir(spark_path))
os.environ['PYSPARK_PYTHON'] = sys.executable
# spark-2.3, spark-2.4
# os.environ['SPARK_HOME'] = os.path.join(spark_path,'spark-2.3')

# use spakr 3.1

os.environ['SPARK_HOME'] = '/home/joetsai/download/spark-3.1.2-bin-hadoop3.2'

You have pyspark version :  ['spark-2.3', 'spark-3.1.2-bin-hadoop2.7', 'spark-3.0', 'spark-3.0.1-bin-hadoop2.7', 'spark-2.3.4-bin-hadoop2.7', 'spark-2.4.7-bin-hadoop2.7', 'spark-2.4', 'spark-3.1']


In [5]:
os.environ['SPARK_HOME']

'/home/joetsai/download/spark-3.1.2-bin-hadoop3.2'

In [6]:
import re
from os.path import join
import pandas as pd
from pyspark.sql import SparkSession as Session
from pyspark.sql import DataFrame
from pyspark import SparkConf as Conf
from pyspark.sql import functions as F, Window as W, types as T
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
C = F.col

In [7]:
conf = (Conf()
    .set('spark.sql.sources.partitionOverwriteMode', 'dynamic')
    .set('spark.driver.memory', '4g')
    .set('spark.driver.maxResultSize', '1g')
   )

In [8]:
spark = (Session
     .builder
     .appName('learn-regex-with-pyspark')
     .master('local[2]')
     .config(conf=conf)
     .getOrCreate())

21/07/31 00:23:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/07/31 00:23:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/07/31 00:23:24 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/07/31 00:23:24 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [9]:
spark

# Regex

## Simple match

斷句 : 從 `<p></p>` or `<br>`

sentence_break = `"</?[Pp]>|<[br|BR]>"`

In [10]:
data = [
'嘉義農會房屋貸款試算(快速貸款 必看推薦)汽車二胎貸款 民間二胎借貸 <br><a href="https://lihi1.cc/FnScd" title="嘉義農會房屋貸款試算" target="_blank">嘉義農會房屋貸款試算</a><BR><a href="https://lihi1.cc/FnScd" title="佳里鎮農會信用貸款陷阱" target="_blank">佳里鎮農會信用貸款陷阱</a><br><a href="https://lihi1.cc/FnScd" title="慶豐銀行房屋貸款試算" target="_blank">慶豐銀行房屋貸款試算</a><br><a href="https://lihi1.cc/FnScd" title="美濃農會房屋貸款條件" target="_blank">美濃農會房屋貸款條件</a><br><a href="https://lihi1.cc/FnScd" title="澎湖區漁會汽車貸款" target="_blank">澎湖區漁會汽車貸款</a><br><a href="https://lihi1.cc/FnScd" title="金門信合社信用貸款是什麼" target="_blank">金門信合社信用貸款是什麼</a><br><a href="https://lihi1.cc/FnScd" title="六甲農會機車貸款流程" target="_blank">六甲農會機車貸款流程</a><br><p>ok 忠訓 ptt借錢OK忠訓國際評價在PTT貸款板是負評OK忠訓國際評價在PTT貸款板上真的是負評嗎ok忠訓ptt忠訓評價pttok<a href="https://lihi1.cc/FnScd" title="嘉義農會房屋貸款試算" target="_blank">嘉義農會房屋貸款試算</a>',

'<P>        台灣股票雜誌2014-9-4</P><P>── ──股票分析及預測（第3299>>>2014-9-4>></P><P>           蘋果公司（AppleInc.）今天邀請媒體9月9日到其加州總部古柏迪諾（Cupertino）參加一項「特別活動」，預料將會推出粉絲期待已久的大螢幕iPhone 6。</P>'
]

indexed_data = [
    (idx, html) for idx, html in enumerate(data)
]

sdf = spark.createDataFrame(indexed_data, ['doc_id','html'])


sentence_break = "</?[Pp]>|<(br)|(BR)>"

(
    sdf
    .withColumn(
                "cleaned", F.regexp_replace(C("html"), fr"{sentence_break}", "\n")
            )
    .select(
            'doc_id',
            'cleaned',
            F.posexplode_outer(F.split("cleaned",'\n')).alias('sentence_id','sentence')
        )
    .withColumn('sentence',F.trim('sentence'))
    .where(F.length('sentence') > 0)
).show(vertical=True,truncate=False)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## 1.2  replace

In [11]:
data = [
'嘉義農會房屋貸款試算(快速貸款 必看推薦)汽車二胎貸款 民間二胎借貸 <br><a href="https://lihi1.cc/FnScd" title="嘉義農會房屋貸款試算" target="_blank">嘉義農會房屋貸款試算</a><BR><a href="https://lihi1.cc/FnScd" title="佳里鎮農會信用貸款陷阱" target="_blank">佳里鎮農會信用貸款陷阱</a><br><a href="https://lihi1.cc/FnScd" title="慶豐銀行房屋貸款試算" target="_blank">慶豐銀行房屋貸款試算</a><br><a href="https://lihi1.cc/FnScd" title="美濃農會房屋貸款條件" target="_blank">美濃農會房屋貸款條件</a><br><a href="https://lihi1.cc/FnScd" title="澎湖區漁會汽車貸款" target="_blank">澎湖區漁會汽車貸款</a><br><a href="https://lihi1.cc/FnScd" title="金門信合社信用貸款是什麼" target="_blank">金門信合社信用貸款是什麼</a><br><a href="https://lihi1.cc/FnScd" title="六甲農會機車貸款流程" target="_blank">六甲農會機車貸款流程</a><br><p>ok 忠訓 ptt借錢OK忠訓國際評價在PTT貸款板是負評OK忠訓國際評價在PTT貸款板上真的是負評嗎ok忠訓ptt忠訓評價pttok<a href="https://lihi1.cc/FnScd" title="嘉義農會房屋貸款試算" target="_blank">嘉義農會房屋貸款試算</a>',

'<P>        台灣股票雜誌2014-9-4</P><P>── ──股票分析及預測（第3299>>>2014-9-4>></P><P>           蘋果公司（AppleInc.）今天邀請媒體9月9日到其加州總部古柏迪諾（Cupertino）參加一項「特別活動」，預料將會推出粉絲期待已久的大螢幕iPhone 6。</P>'
]

indexed_data = [
    (idx, html) for idx, html in enumerate(data)
]

sdf = spark.createDataFrame(indexed_data, ['doc_id','html'])


sentence_break = "</?[Pp]>|<(br)|(BR)>"

# sometimes you need use it in udf

def python_replace(html : str) -> str:
    return re.sub(r'https://',r'http://',html)
    
(
    sdf
    .withColumn(
                "cleaned", F.regexp_replace(C("html"), fr"{sentence_break}", "\n")
            )
    .select(
            'doc_id',
#             'cleaned',
            F.posexplode_outer(F.split("cleaned",'\n')).alias('sentence_id','sentence')
        )
    .withColumn('sentence',F.trim('sentence'))
    .withColumn('sentence',F.udf(python_replace,'string')('sentence'))
    .where(F.length('sentence') > 0)
).show(vertical=True,truncate=False)

-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 doc_id      | 0                                                                                                                                                                                                                
 sentence_id | 0                                                                                                                                                                                                                
 sentence    | 嘉義農會房屋貸款試算(快速貸款 必看推薦)汽車二胎貸款 民間二胎借貸                                                                                                                                                 
-RECORD 1-------------------------------------------------------------------------------------------------------------------------

## Negtive ahead

## non-greedy match

`?` : match 0 or 1
`+` : match 1 or more
`*` : match 0 or more

以上是 Greedy mode，也就是說會盡可能多的match

例如 

html = `<a href="http://www.flickr.com/photos/126291387@N06/51051131972" target="_blank"><img alt="DSC01611" src="//live.staticflickr.com/65535/51051131972_100386c21c_b.jpg" title=""></a><a href="http://www.flickr.com/photos/126291387@N06/51051047806" target="_blank"><img alt="DSC01610" src="//live.staticflickr.com/65535/51051047806_7b2587931f_b.jpg" title=""></a>`

pattern = `"<img.+['\"]>"` - match 盡可能多的文字，在以上例子中會match到超過1個的imgtag

Non-greedy mode

`??` : match 0 or 1
`+?` : match 1 or more
`*?` : match 0 or more

pattern = `"<img.+?['\"]>"` - match 盡可能少的文字，在此例子中僅會match到1個imgtag

demo : 

https://regex101.com/r/5QEqLQ/1


In [12]:

data = [
'<a href="http://www.flickr.com/photos/126291387@N06/51051131972" target="_blank"><IMG alt="DSC01611" src="//live.staticflickr.com/65535/51051131972_100386c21c_b.jpg" title=""></a><a href="http://www.flickr.com/photos/126291387@N06/51051047806" target="_blank"><IMG alt="DSC01610" src="//live.staticflickr.com/65535/51051047806_7b2587931f_b.jpg" title=""></a>',

    '''<p style="text-align: center;" align="center">     <strong><span style="font-size: 18pt; font-family: 'arial black', 'avant garde'; color: #ff0000;"><img src="//s.pixfs.net/f.pixnet.net/images/emotions/006.gif" alt="" border="0" />前 言<img src="//s.pixfs.net/f.pixnet.net/images/emotions/006.gif" alt="" border="0" /></span></strong>''',
    '<p style="margin: 0px 0px 1em; padding: 0px; font-size: 13px; color: #666666; font-family: arial; line-height: 26px;"><span style="color:#333300; font-size:large"><strong><span style="font-size:18pt"><span style="font-size:14pt"><span style="color:#000000">人生難免會遇到要</span><span style="color:#cc3399">買車、買房、創業、周轉、現金需求‧‧‧</span></span><span style="color:#000000"><span style="font-size:14pt">的時候</span> </span></span><span style="color:#000000"><img alt="" height="47" src="http://0.share.photo.xuite.net/brack2381/10e91cc/19080521/1038628227_M.jpg" style="vertical-align: middle; display: inline;" title="" width="47" />',
    '<)，一入口感受到的超~~~軟嫩又Juicy的鴨肉，整個鴨肉湯汁在口腔裡噴發，超愛！(果然是志玲姊姊最愛的金賓料理<img alt="" height="19" src="//s.pixfs.net/f.pixnet.net/images/emotions/omg_smile.gif" title="" width="19">'
]

indexed_data = [
    (idx, html) for idx, html in enumerate(data)
]

sdf = spark.createDataFrame(indexed_data, ['doc_id','html'])

imgtag = "<[(img)|(IMG)].+?>"
(
    sdf
    .withColumn("html", F.regexp_replace(C("html"), fr"{imgtag}", "\n$0\n"))
    .select(
        'doc_id',
        F.posexplode_outer(F.split("html",'\n')).alias('sentence_id','imgtag')
    )
).show(vertical=True,truncate=False)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 doc_id      | 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
 sentence_id | 0          

# html

In [13]:
# replace wired symbol in html

import html


df = (
    spark.createDataFrame([(0,'&gt;'),(1,'&nbsp;')],['id','content'])
    .withColumn("cleaned", F.udf(html.unescape)("content"))
)

df.show()

+---+-------+-------+
| id|content|cleaned|
+---+-------+-------+
|  0|   &gt;|      >|
|  1| &nbsp;|       |
+---+-------+-------+



# bs4

In [14]:
from bs4 import BeautifulSoup

In [15]:

# print(soup.prettify())

## `<imgsrc ... >`

In [16]:

from typing import List

In [17]:
# imgs
data = ['<pstyle="text-align:left;"><imgsrc="https://pic.pimg.tw/peko721/1555405696-3356669416.jpg"alt="麥食達韓式料理.麥食達菜單.麥食達.台北車站美食.北車美食.石鍋拌飯."/>',
'<img alt="IMG_5219.jpg" src="https://pic.pimg.tw/happy78/1528543949-4214686345_n.jpg" title="IMG_5219.jpg">',
'<)，一入口感受到的超~~~軟嫩又Juicy的鴨肉，整個鴨肉湯汁在口腔裡噴發，超愛！(果然是志玲姊姊最愛的金賓料理<img alt="" height="19" src="//s.pixfs.net/f.pixnet.net/images/emotions/omg_smile.gif" title="" width="19">'
]


indexed_data = [
    (idx, html) for idx, html in enumerate(data)
]

sdf = spark.createDataFrame(indexed_data, ['doc_id','html'])

def get_img(html : str) -> List[str]:
    soup = BeautifulSoup(html, 'html.parser')
    imgtag = soup.findAll('img')
    imgtag2 = soup.findAll('imgsrc')
    print(imgtag, imgtag2)
    
get_img(data[0]), get_img(data[1]), get_img(data[2])

[] []
[<img alt="IMG_5219.jpg" src="https://pic.pimg.tw/happy78/1528543949-4214686345_n.jpg" title="IMG_5219.jpg"/>] []
[<img alt="" height="19" src="//s.pixfs.net/f.pixnet.net/images/emotions/omg_smile.gif" title="" width="19"/>] []


(None, None, None)

## `insert_before, insert_after`

sometimes you just can't extract all the imgtags by `regex`

then use bs4 

https://regex101.com/r/gDUzQs/1/

In [18]:
data = ['<pstyle="text-align:left;"><imgsrc="https://pic.pimg.tw/peko721/1555405696-3356669416.jpg"alt="麥食達韓式料理.麥食達菜單.麥食達.台北車站美食.北車美食.石鍋拌飯."/>',
'<img alt="IMG_5219.jpg" src="https://pic.pimg.tw/happy78/1528543949-4214686345_n.jpg" title="IMG_5219.jpg"><img alt="IMG_5219.jpg" src="https://pic.pimg.tw/happy78/1528543949-4214686345_n.jpg" title="IMG_5219.jpg">',
'<)，一入口感受到的超~~~軟嫩又Juicy的鴨肉，整個鴨肉湯汁在口腔裡噴發，超愛！(果然是志玲姊姊最愛的金賓料理<img alt="" height="19" src="//s.pixfs.net/f.pixnet.net/images/emotions/omg_smile.gif" title="" width="19">'
]


indexed_data = [
    (idx, html) for idx, html in enumerate(data)
]

sdf = spark.createDataFrame(indexed_data, ['doc_id','html'])

def make_imgtag_seperator(html : str) -> List[str]:
    # TODO insert \n to every image tag
    soup = BeautifulSoup(html, 'html.parser')
    imgtag = soup.find_all('img', src=True)
    for tag in imgtag:
        tag.insert_before('\n')
        tag.insert_after('\n')
    return str(soup)
 
    
# imgtag = "<[(img)|(IMG)].+?>"
# (
#     sdf
#     .withColumn("html", F.regexp_replace(C("html"), fr"{imgtag}", "\n$0\n"))
#     .select(
#         'doc_id',
#         F.posexplode_outer(F.split("html",'\n')).alias('sentence_id','imgtag')
#     )
    
(
    sdf
    .withColumn("cleaned", F.regexp_replace(C("html"), fr"{imgtag}", "\n$0\n"))
    .withColumn("cleaned", F.explode_outer(F.split(C("cleaned"), '\n')))
    .withColumn('cleaned',F.udf(make_imgtag_seperator, 'string')('cleaned'))
    .select(
        'doc_id',
        F.posexplode_outer(
            F.split(C("cleaned"),'\n')
        ).alias('sentence_id','sentence')
    )
    .where(F.length(C("sentence")) > 0)
).show(vertical=True, truncate=False)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 doc_id      | 0                                                                                                                                                            
 sentence_id | 0                                                                                                                                                            
 sentence    | <pstyle="text-align:left;"></pstyle="text-align:left;">                                                                                                      
-RECORD 1-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 doc_id      | 0                                                                                                                       